In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("CKD.csv")

In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [4]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [5]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]

In [6]:
dep=dataset[['classification_yes']]

In [7]:
dataset['classification_yes'].value_counts()

classification_yes
True     249
False    150
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
param_grid={'criterion':['gini','entropy','log_loss'],'splitter':['best','random'],
'max_features':['auto','sqrt','log2']}
grid=GridSearchCV(DecisionTreeClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')

In [10]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\NANDHAKUMAR GK\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
30 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\NANDHAKUMAR GK\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\NANDHAKUMAR GK\anaconda3\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\NANDHAKUMAR GK\anaconda3\Lib\site-packages\sklearn\base.py"

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [11]:
grid_prediction=grid.predict(x_test)

In [12]:
re=grid.cv_results_

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)

from sklearn.metrics import classification_report
clf_re=classification_report(y_test,grid_prediction)

In [13]:
print(cm)

[[51  0]
 [ 3 79]]


In [14]:
print(clf_re)

              precision    recall  f1-score   support

       False       0.94      1.00      0.97        51
        True       1.00      0.96      0.98        82

    accuracy                           0.98       133
   macro avg       0.97      0.98      0.98       133
weighted avg       0.98      0.98      0.98       133



In [15]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print("The F1_macro Value of the Best Parameter {}:".format(grid.best_params_),f1_macro)

The F1_macro Value of the Best Parameter {'criterion': 'entropy', 'max_features': 'sqrt', 'splitter': 'random'}: 0.9775556904684072


In [16]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

np.float64(0.9817073170731707)

In [17]:
table=pd.DataFrame.from_dict(re)

In [18]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.058399,0.045084,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
1,0.103891,0.044513,0.000000,0.000000,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
2,0.306802,0.053885,0.457972,0.047095,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.945100,0.903610,0.962573,0.961826,0.981217,0.950865,0.026248,9
3,0.272067,0.057133,0.444556,0.053168,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.963284,0.962264,0.962573,0.981217,1.000000,0.973868,0.014907,3
4,0.216017,0.053798,0.414138,0.087985,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.926978,0.943699,0.868519,0.961826,0.922492,0.924703,0.031323,11
5,0.256948,0.070445,0.448204,0.053379,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.944135,0.942166,1.000000,0.922492,0.981031,0.957965,0.028278,6
6,0.051089,0.028917,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
7,0.075644,0.026999,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
8,0.230443,0.094771,0.384241,0.022718,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.926978,0.924528,0.981217,0.942332,0.961826,0.947376,0.021529,10
9,0.214632,0.045802,0.322022,0.067994,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",1.000000,0.944161,0.981217,0.981217,1.000000,0.981319,0.020390,1
